In [8]:
import PyPDF2
import fitz
from pdf2image import convert_from_path, convert_from_bytes

from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

In [9]:

def convert_pdf(path):
   l=path.split('\\')
   file=l[-1]
   pdf_name=file.split('.')[0]
   #print(pdf_name)
   images=convert_from_path(path)

   save_path="Data\\Converted_images\\"
   n=0
   for i,image in enumerate(images):
       fname = "image_"+pdf_name+"_"+ str(i) + ".png"
       image.save(save_path+fname, "PNG")
       n=n+1
   return n

def is_scanned_pdf(file_path):
    """
    Determines if a PDF file is scanned or not by checking if it contains text or not.

    Parameters:
        file_path (str): The path to the PDF file to check.

    Returns:
        bool: True if the PDF file is scanned, False otherwise.
    """
    with fitz.open(file_path) as pdf:
        for page in pdf:
            text = page.get_text()
            if text.strip() != "":
                # If the page contains text, it's not a scanned PDF
                return False
    # If all pages don't contain text, it's a scanned PDF
    return True

def is_pdf_text_based(pdf_file_path):
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Check if PDF file is encrypted
        if pdf_reader.is_encrypted:
            return False

        # Check each page in the PDF file
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]

            # Check if page contains any images
            if '/XObject' in page['/Resources']:
                return False

            # Check if page contains any text
            if '/Font' in page['/Resources'] or '/Text' in page['/Contents']:
                return True

        # If PDF file contains no pages, return False
        return False


In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
pip install googletrans==3.1.0a0


[notice] A new release of pip available: 22.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
from googletrans import Translator

translator = Translator()

In [13]:
import PyPDF2
#from googletrans import Translator
import numpy as np
import cv2
from PIL import Image
import pytesseract

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'


def ocr_core(image):
    text = pytesseract.image_to_string(image, lang='fra', config='-c page_separator=""')
    return text


def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)


def remove_noise(image):
    return cv2.medianBlur(image, 5)


def thresholding(image):
    # need thrershold optimization !!!!!!!@TODO/TO SEARCH
    return cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)


def ocr_extract_txt(image_name):
    img = cv2.imread('Data\\Converted_images\\' + image_name)
    img = cv2.resize(img, (0, 0), fx=0.9, fy=0.9)
    # img = np.invert(img)
    # img=get_grayscale(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # img=thresholding(img)[1]
    #print(ocr_core(img))
    # img2=get_grayscale(img2)
    #cv2.imshow('Image', img)
    #cv2.waitKey(0)

    # Find the contours in the image and store them in an array
    # contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # text_contours = cv2.drawContours(img, contours, -1, (0, 255, 0), 3)

    # Show the contours
    # cv2.imshow('Contours', text_contours)
    # cv2.waitKey(0)
    # text = pytesseract.image_to_string(img, lang='eng')
    text_file = open("Data\\Converted_images\\AllText.txt", "w")
    text_file.write(ocr_core(img[1]))
    return ocr_core(img)



def extract_text_from_pdf(pdf_file_path):
    text = ""

    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # Iterate through each page in the PDF file
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]

            # Extract text from the page
            page_text = page.extract_text()

            # Append the page text to the overall text string
            text += page_text

    return text


def extract(pdf_file_path):
    if is_scanned_pdf(pdf_file_path):
        convert_pdf(pdf_file_path)
        l = pdf_file_path.split('\\')
        file = l[-1]
        pdf_name = file.split('.')[0]
        t=""
        for i in range(convert_pdf(pdf_file_path)):
            fname = "image_" + pdf_name + "_" + str(i) + ".png"
            #print(fname)
            t=t+ocr_extract_txt(fname)
        return t    
            
        
    else :
        return extract_text_from_pdf(pdf_file_path)


## extract is  a complete function


In [15]:

lop=extract('data\\ATS.pdf').split('\n')
print(lop)

['Catalogue des sujets', 'de stage de PFE 2023', '2 Stages PFE 2023Qui sommes -nous', 'Avec une présence en France et en Tunisie,  ', 'ATS se place aujourd’hui comme spécialiste du ', 'digital intervenant dès les étapes d’idéation,', 'de stratégie et de conception jusqu’à ', 'l’implémentation d’applications répondant aux ', 'enjeux de ses clients internationaux.Développement ', 'Web / MobileConseil Digital', 'Ops & ', 'Cybersecurity', 'QA', '3ATS en quelques chiffres', 'Âge moyen28 ', 'ansPersonnes dans', 'l’équipe', '20 15 30%', 'ClientsTaux de ', 'croissance', 'Stages PFE 2023', '4Avantages pour les stagiaires ', 'de PFE 2022', 'IndemnitésEntraidePartage de ', 'connaissances', 'Tickets ', 'restaurantsStage de pré -', 'embauche', 'Stages PFE 2023', '5Comment postuler', "Afin de postuler, ilvous suffit d'envoyer un email àl'adresse", ':recrutement@ats -digital .com avec :', '•Objet du mail :[Candidature Stage] -référence de stage -', 'NOM Prénom', '•Mettre CV etLM au format PDF en pièc

### Cleaning the texg

In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [17]:
filtered_lop=[string for string in lop if string.strip() and not string.strip().isdigit()]
#print(filtered_lop)
def st(a):
    return a.strip()
cleaned_lop=list(map(st,filtered_lop))
#print(cleaned_lop)
raw_txt="".join(cleaned_lop)
print(raw_txt)

Catalogue des sujetsde stage de PFE 20232 Stages PFE 2023Qui sommes -nousAvec une présence en France et en Tunisie,ATS se place aujourd’hui comme spécialiste dudigital intervenant dès les étapes d’idéation,de stratégie et de conception jusqu’àl’implémentation d’applications répondant auxenjeux de ses clients internationaux.DéveloppementWeb / MobileConseil DigitalOps &CybersecurityQA3ATS en quelques chiffresÂge moyen28ansPersonnes dansl’équipe20 15 30%ClientsTaux decroissanceStages PFE 20234Avantages pour les stagiairesde PFE 2022IndemnitésEntraidePartage deconnaissancesTicketsrestaurantsStage de pré -embaucheStages PFE 20235Comment postulerAfin de postuler, ilvous suffit d'envoyer un email àl'adresse:recrutement@ats -digital .com avec :•Objet du mail :[Candidature Stage] -référence de stage -NOM Prénom•Mettre CV etLM au format PDF en pièces jointes•Fichier CV ànommer :CV_NOM_Prenom .pdf•Fichier LM ànommer :LM_NOM_Prenom .pdfIlest aussi possible de postuler directement sur notre siteint

In [ ]:
pip install spacy



[notice] A new release of pip available: 22.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!python -m spacy download en_core_web_lg

     -------------------------------------- 587.7/587.7 MB 1.6 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [20]:
pip install skillNer

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for skillNer: filename=skillNer-1.0.3-py3-none-any.whl size=25648 sha256=6ee9563f059a3daf5a364092af77cf528a8ec4d221f5e16e78cd3c7e43fffe44
  Stored in directory: c:\users\msi gf63\appdata\local\pip\cache\wheels\eb\a1\1a\735667a07483c24897ef0857e0dcbc9bd9d2c0cd0c840c7d7d
Successfully built skillNer

[notice] A new release of pip available: 22.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
# imports
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

# extract skills from job_description
job_description = """
You are a Python developer with a solid experience in web development
and can manage projects. You quickly adapt to new environments
and speak fluently English and French
"""

annotations = skill_extractor.annotate(job_description)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [24]:
print(annotations)

{'text': 'you are a python developer with a solid experience in web development and can manage projects you quickly adapt to new environments and speak fluently english and french', 'results': {'full_matches': [{'skill_id': 'KS122Z36QK3N5097B5JH', 'doc_node_value': 'web development', 'score': 1, 'doc_node_id': [10, 11]}], 'ngram_scored': [{'skill_id': 'KS125LS6N7WP4S6SFTCK', 'doc_node_id': [3], 'doc_node_value': 'python', 'type': 'fullUni', 'score': 1, 'len': 1}, {'skill_id': 'KS1267F6MSPN366LX7ST', 'doc_node_id': [14, 15], 'doc_node_value': 'manage projects', 'type': 'lowSurf', 'score': 2, 'len': 2}, {'skill_id': 'KS120626HMWCXJWJC7VK', 'doc_node_id': [18], 'doc_node_value': 'adapt', 'type': 'lowSurf', 'score': 0.5245310068130493, 'len': 1}, {'skill_id': 'KS123K75YYK8VGH90NCS', 'doc_node_id': [25], 'doc_node_value': 'english', 'type': 'lowSurf', 'score': 1, 'len': 1}, {'skill_id': 'KS1243976G466GV63ZBY', 'doc_node_id': [27], 'doc_node_value': 'french', 'type': 'lowSurf', 'score': 1, '

In [12]:
pip install find_job_titles

  Using cached find_job_titles-0.7.0-py2.py3-none-any.whl (383 kB)
  Using cached pyahocorasick-2.0.0-cp39-cp39-win_amd64.whl (39 kB)
  Using cached acora-2.3.tar.gz (215 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for acora: filename=acora-2.3-cp39-cp39-win_amd64.whl size=117543 sha256=ca3ba64737b0d59629647e3b49af24675efe5dfb23963a9d009cc617351b2484
  Stored in directory: c:\users\msi gf63\appdata\local\pip\cache\wheels\99\bf\f7\75667d77774e3b5e57b17b03489c7b14037b9c416940cfc953
Successfully built acora
Note: you may need to restart the kernel to use updated packages.


## TEST SPACY !!!!!!!!!!!!!!!!!!!!

In [3]:
!python -m spacy download en_core_web_sm

import spacy 

nlp = spacy.load("en_core_web_sm")
nlp.pipe_names

     -------------------------------------- 12.8/12.8 MB 930.7 kB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [6]:
sentence = "Daniil Medvedev and Novak Djokovic have built an intriguing rivalry since the Australian Open decider, which the Serb won comprehensively."
doc = nlp(sentence)

from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

[(X, X.ent_iob_, X.ent_type_) for X in doc if X.ent_type_]

[(Daniil, 'B', 'PERSON'),
 (Medvedev, 'I', 'PERSON'),
 (Novak, 'B', 'PERSON'),
 (Djokovic, 'I', 'PERSON'),
 (Australian, 'B', 'NORP'),
 (Serb, 'B', 'NORP')]

In [10]:
pip install --upgrade setuptools

     ---------------------------------------- 1.1/1.1 MB 2.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 61.2.0
    Uninstalling setuptools-61.2.0:
      Successfully uninstalled setuptools-61.2.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.


In [18]:
from find_job_titles import FinderAcora
finder=FinderAcora()
print(finder.findall(u"Objective.Description of the young talent positions that we would like to share with universities,disrupted schools and vocational education centres, showing requirements and valueproposition offered by Santander Corporate & Investment Banking (SCIB).Document structure.- Common Santander T&O value proposition.- Profiles description:o DevSecOps.o Software engineering.o Data development and solutions.o Cybersecurity analyst."))

RuntimeError: generator raised StopIteration